In [ ]:
#Import packages

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
import math
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn import linear_model, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.cross_validation import cross_val_predict
from sklearn.svm.libsvm import cross_validation
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import data
rep_data = pd.read_excel("data_stedin_excel_min.xlsx")
# rep_data = rep_data[rep_data['aangepakt'] == rep_data['aangepakt']]

# Remove irrelevant columns
columns = ['woningtype_naam',
       'woningtype_bouwperiode', 'woningtype_daktype', 'aangepakt', 'SJV_2017_ELEKTRA', 'SJV_2017_GAS', 'categorie_match', 'pc6']

rep_data = rep_data[columns]

# Create data backup
rep_data_copy = rep_data.copy()

In [ ]:
# Data preparation

# Import second data set
data = pd.read_excel("postcodes2017.xlsx")
data = data.set_index("PC4,N,5,0")

columns = ['INWONER,N,10,0', 'MAN,N,10,0', 'VROUW,N,10,0',
       'INW_014,N,10,0', 'INW_1524,N,10,0', 'INW_2544,N,10,0',
       'INW_4564,N,10,0', 'INW_65PL,N,10,0',
       'P_NL_ACHTG,N,10,0', 'P_WE_MIG_A,N,10,0', 'P_NW_MIG_A,N,10,0',
       'AANTAL_HH,N,10,0', 'TOTHH_EENP,N,10,0', 'TOTHH_MPZK,N,10,0',
       'HH_EENOUD,N,10,0', 'HH_TWEEOUD,N,10,0', 'GEM_HH_GR,N,19,11',
       'WONING,N,10,0', 'WONVOOR45,N,10,0', 'WON_4564,N,10,0',
       'WON_6574,N,10,0', 'WON_7584,N,10,0', 'WON_8594,N,10,0',
       'WON_9504,N,10,0', 'WON_0514,N,10,0',
       'WON_MRGEZ,N,10,0',
       'WON_HCORP,N,10,0', 'STED,N,10,0']

data = data[columns]

# Remove missing data
for i in data.columns:
    data = data[data[i] != -99997]

# Create percentage columns    
data['PERC_EENP'] = data['TOTHH_EENP,N,10,0'] / data['AANTAL_HH,N,10,0'] * 100
data['PERC_MPZK'] = data['TOTHH_MPZK,N,10,0'] / data['AANTAL_HH,N,10,0'] * 100
data['PERC_EENOUD'] = data['HH_EENOUD,N,10,0'] / data['AANTAL_HH,N,10,0'] * 100
data['PERC_TWEEOUD'] = data['HH_TWEEOUD,N,10,0'] / data['AANTAL_HH,N,10,0'] * 100

data['PERC_MAN'] = data['MAN,N,10,0'] / data['INWONER,N,10,0'] * 100
data['PERC_VROUW'] = data['VROUW,N,10,0'] / data['INWONER,N,10,0'] * 100
data['PERC_014'] = data['INW_014,N,10,0'] / data['INWONER,N,10,0'] * 100
data['PERC_1524'] = data['INW_1524,N,10,0'] / data['INWONER,N,10,0'] * 100
data['PERC_2544'] = data['INW_2544,N,10,0'] / data['INWONER,N,10,0'] * 100
data['PERC_4564'] = data['INW_4564,N,10,0'] / data['INWONER,N,10,0'] * 100
data['PERC_65'] = data['INW_65PL,N,10,0'] / data['INWONER,N,10,0'] * 100

data['TOTAAL_WON'] = data['WONVOOR45,N,10,0'] + data['WON_4564,N,10,0'] + data['WON_6574,N,10,0'] + data['WON_7584,N,10,0'] + data['WON_8594,N,10,0'] + data['WON_9504,N,10,0'] + data['WON_0514,N,10,0']

data['PERC_WON_045'] = data['WONVOOR45,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_4564'] = data['WON_4564,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_6574'] = data['WON_6574,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_7584'] = data['WON_7584,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_8594'] = data['WON_8594,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_9504'] = data['WON_9504,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_0514'] = data['WON_0514,N,10,0'] / data['TOTAAL_WON'] * 100

columns = ['P_NL_ACHTG,N,10,0',
       'P_WE_MIG_A,N,10,0', 'P_NW_MIG_A,N,10,0',
       'PERC_EENP', 'PERC_MPZK', 'PERC_EENOUD', 'PERC_TWEEOUD', 'GEM_HH_GR,N,19,11',
       'STED,N,10,0', 'PERC_MAN', 'PERC_014', 'PERC_1524', 'PERC_2544',
       'PERC_4564', 'PERC_WON_045', 'PERC_WON_4564',
       'PERC_WON_6574', 'PERC_WON_7584', 'PERC_WON_8594', 'PERC_WON_9504',
       'PERC_WON_0514', 'WON_HCORP,N,10,0', 'AANTAL_HH,N,10,0']

data = data[columns]

data.columns = ['NL_ACHT', 'WE_ACHT', 'NW_ACHT', 'EENP', 'MPZK', 'EENOUD', 'TWEEOUD', 'HH_GR', 'STED', 'MAN',
              'L014', 'L1425', 'L2544', 'L4564', 'H045', 'H4564', 'H6574', 'H7584', 'H8594', 'H9504', 'H0514', 'WON_HCORP', 'AANTAL_HH']

cbs_data = data.copy()

# Identify PC4's covered by both data sets
rep_data['PC4'] = rep_data['pc6']
rep_data['PC4'] = rep_data['PC4'].apply(lambda x: x[0:4])

k = rep_data['PC4'].unique()
for i in range(len(k)):
    k[i] = int(k[i])
    
k = set(k)
    
l = set(cbs_data.index.unique())

postcodes = l.intersection(k)

cbs_data = cbs_data.loc[postcodes, :]

# Create data backup
cbs_data_copy = cbs_data.copy()

In [ ]:
def create_combi(rep_data, cbs_data, quantile, classes):    
    # Create percentage inefficiency variable
    cutoff = rep_data['SJV_2017_ELEKTRA'].quantile(quantile)
    
    rep_data['inef_elek'] = rep_data['SJV_2017_ELEKTRA']
    rep_data['inef_elek'] = rep_data['inef_elek'].apply(lambda x: (x > cutoff) * 1)
    
    post_occurences = dict(rep_data['PC4'].value_counts())
    
    rep_data['PC4'] = rep_data['PC4'].apply(lambda x: int(x))

    # Filter postal codes
    cbs_data = cbs_data.loc[postcodes2, :]
    
    # Fill in percentage inefficiency variable for CBS data set
    cbs_data['inef'] = 0

    for i in postcodes2:
        temp = rep_data[rep_data['PC4'] == i]
        inef = temp['inef_elek'].mean()
        cbs_data.loc[i, 'inef'] = inef

    cbs_data['inef_bin'] = cbs_data['inef']
    
    # Create binary variable based on cut-off
    cutoff2 = cbs_data['inef'].quantile(classes)
    cbs_data['inef_bin'] = cbs_data['inef_bin'].apply(lambda x: (x > cutoff2) * 1)
    
    return cbs_data

In [ ]:
def analyse(cbs_data, seed, over, j):
    # This function predict whether a cluster has a high amount of inefficient homes using four classification
    # models: KNN, RF, SVM and NN. Also, it offers the possibility to include 2 different resampling methods
    
    random.seed(seed)
    
    x_columns = ['NL_ACHT', 'WE_ACHT', 'NW_ACHT', 'EENP', 'MPZK', 'EENOUD', 'TWEEOUD', 'HH_GR', 'STED', 'MAN',
              'L014', 'L1425', 'L2544', 'L4564', 'H045', 'H4564', 'H6574', 'H7584', 'H8594', 'H9504', 'H0514']

    y_column = ['inef_bin']

    X = cbs_data[x_columns]
    # X = pd.get_dummies(X)
    y = cbs_data[y_column]
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    # Reduce size of efficient class through undersampling
    if (over == 0) and (j != 0.5):
        X_train = pd.DataFrame(X_train)
        X_train.index = y_train.index
        y_train_inef = y_train[y_train['inef_bin'] == 1]
        y_train_ef = y_train[y_train['inef_bin'] == 0]
        X_train_inef = X_train.loc[y_train_inef.index, :]

        #Undersampling
        from random import sample 

        # Prints list of random items of given length 
        list1 = set(y_train_ef.index)

        samp = sample(list1, len(y_train_inef))

        y_train_ef = y_train.loc[samp, :]
        X_train_ef = X_train.loc[samp, :]
        
        y_train = pd.concat([y_train_ef, y_train_inef])
        X_train = pd.concat([X_train_ef, X_train_inef])
    
    # Increase size of inefficient class through oversampling
    if (over == 1) and (j != 0.5):
        # transform the dataset
        X_train, y_train = SMOTE().fit_sample(X_train, y_train)
    
    # KNN
    neigh = [1, 2, 3, 4, 5, 6, 7]
    random_grid = {'n_neighbors': neigh}

    knn_random = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = random_grid, cv = 5, verbose=0, n_jobs = -1)

    knn_random.fit(X_train, y_train)

    knn_predictions = knn_random.predict(X_test)
    
    # SVM
    C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    gammas = [0.001, 0.01, 0.1, 1, 10, 100]
    
    # Create the random grid
    random_grid = {'C': C,'gamma': gammas}

    SVM_random = GridSearchCV(estimator = SVC(), param_grid = random_grid, cv = 5, verbose=0, n_jobs = -1)
    
    SVM_random.fit(X_train, y_train)
    svm_predictions = SVM_random.predict(X_test)
    
    # RF
    rf = RandomForestClassifier(n_estimators = 100)
    rf.fit(X_train, y_train)
    rf_predictions = rf.predict(X_test)
    
    # NN
    mlp_gs = MLPClassifier(max_iter=100)
    
    import itertools
    x = [0, 5, 10, 20]
    sizes = [p for p in itertools.product(x, repeat=4)]
    
    for i in range(len(sizes)):
        p = list(sizes[i])
        while 0 in p:
            p.remove(0)
            sizes[i] = p
    
    parameter_space = {
        'hidden_layer_sizes': sizes,
        'activation': ['relu'],
        'solver': ['lbfgs'],
        'alpha': [1e-5],
        'learning_rate': ['constant'],
    }
    
    clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
    clf.fit(X, y)
    
    nn_predictions = clf.predict(X_test)
    
    # Create accuracy scores for algorithms above
    trues = list(y_test['inef_bin'])

    errors = [abs(a_i - b_i) for a_i, b_i in zip(knn_predictions, trues)]

    rmse = [math.sqrt(x**2) for x in errors]

    knn_rmse = 1 - sum(rmse)/len(rmse)

    errors = [abs(a_i - b_i) for a_i, b_i in zip(rf_predictions, trues)]

    rmse = [math.sqrt(x**2) for x in errors]

    rf_rmse = 1 - sum(rmse)/len(rmse)

    errors = [abs(a_i - b_i) for a_i, b_i in zip(svm_predictions, trues)]

    rmse = [math.sqrt(x**2) for x in errors]

    svm_rmse = 1 - sum(rmse)/len(rmse)

    errors = [abs(a_i - b_i) for a_i, b_i in zip(nn_predictions, trues)]

    rmse = [math.sqrt(x**2) for x in errors]

    nn_rmse = 1 - sum(rmse)/len(rmse)
    
    # Create recall scores for all 4 algorithms
    from sklearn.metrics import recall_score
    knn_rec = recall_score(trues, knn_predictions)
    rf_rec = recall_score(trues, rf_predictions)
    svm_rec = recall_score(trues, svm_predictions)
    nn_rec = recall_score(trues, nn_predictions)
    
    # Create accuracy scores if only predicting 0 or 1
    preds = [1] * len(y_test)
    trues = y_test[y_column]
    trues = list(trues.iloc[:, 0])
    
    cols = ['trues', 'preds']
    errors = pd.DataFrame(columns=cols)
    errors['trues'] = trues
    errors['preds'] = preds
    
    errors['errors'] = errors['trues'] - errors['preds']

    errors['errors'] = errors['errors'].apply(lambda x: math.sqrt(x**2))
    
    true_rmse = 1 - sum(errors['errors'])/len(errors)
    false_rmse = 1 - true_rmse
    
    return [round(knn_rmse, 2), round(rf_rmse, 2), round(svm_rmse, 2), round(nn_rmse, 2), round(false_rmse, 2), round(true_rmse, 2)], [round(knn_rec, 2), round(rf_rec, 2), round(svm_rec, 2), round(nn_rec, 2)]

In [ ]:
# Initialize cut-off values
cut = [0.5, 0.6, 0.7, 0.8]
cut2 = [0.5, 0.6, 0.7, 0.8]

# Initialize common seeds for all methods
seeds = []

index = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

import random

for i in range(10):
    seeds.append(random.randint(0, 1000))

for i in cut:
    for j in cut2:
        
            print(i, j)
            
            # Start analysis without resampling method
            print('Normaal')
            
            # Initialize empty dataframes for storage of scores
            columns = ['KNN', 'RF', 'SVM', 'NN', '0', '1']
            columns2 = ['KNN', 'RF', 'SVM', 'NN']
            
            results_rmse = pd.DataFrame(columns=columns)
            results_recall = pd.DataFrame(columns=columns2)
            
            data = create_combi(rep_data.copy(), cbs_data.copy(), i, j)
            
            # Start analysis
            for s in seeds:
                # Format scores
                rmse, recall = analyse(data, s, 10, j)
                temp2 = pd.DataFrame(rmse)
                temp3 = temp2.transpose()
                temp3.columns = columns
                results_rmse = results_rmse.append(temp3)
                    
                temp2 = pd.DataFrame(recall)
                temp3 = temp2.transpose()
                temp3.columns = columns2
                results_recall = results_recall.append(temp3)
            
            results_rmse.index = index
            results_recall.index = index
            
            temp = pd.DataFrame(0, index=['Average'], columns=columns)
            for n in results_rmse.columns:
                temp.loc['Average', n] = results_rmse[n].mean()
            results_rmse = pd.concat([results_rmse, temp])
            
            temp = pd.DataFrame(0, index=['Average'], columns=columns2)
            for n in results_recall.columns:
                temp.loc['Average', n] = results_recall[n].mean()
            results_recall = pd.concat([results_recall, temp])
            
            # Save stores to csv file
            results_rmse.to_csv(str(i) + str(j) + 'rmse_mean.csv')
            results_recall.to_csv(str(i) + str(j) + 'recall_mean.csv')
            
            # Initialize empty dataframes for storage of scores
            results_rmse = pd.DataFrame(columns=columns)
            results_recall = pd.DataFrame(columns=columns2)
            
            # Start analysis for undersampling
            print('Under')
            for s in seeds:
                # Format scores
                rmse, recall = analyse(data, s, 0, j)
                temp2 = pd.DataFrame(rmse)
                temp3 = temp2.transpose()
                temp3.columns = columns
                results_rmse = results_rmse.append(temp3)
                    
                temp2 = pd.DataFrame(recall)
                temp3 = temp2.transpose()
                temp3.columns = columns2
                results_recall = results_recall.append(temp3)

            results_rmse.index = index
            results_recall.index = index                
                
            temp = pd.DataFrame(0, index=['Average'], columns=columns)
            for n in results_rmse.columns:
                temp.loc['Average', n] = results_rmse[n].mean()
            results_rmse = pd.concat([results_rmse, temp])
            
            temp = pd.DataFrame(0, index=['Average'], columns=columns2)
            for n in results_recall.columns:
                temp.loc['Average', n] = results_recall[n].mean()
            results_recall = pd.concat([results_recall, temp])
            
            # Save stores to csv file
            results_rmse.to_csv(str(i) + str(j) + 'rmse_under_mean.csv')
            results_recall.to_csv(str(i) + str(j) + 'recall_under_mean.csv')
            
            # Initialize empty dataframes for storage of scores
            results_rmse = pd.DataFrame(columns=columns)
            results_recall = pd.DataFrame(columns=columns2)
            
            # Start analysis for oversampling
            print('Over')
            for s in seeds:
                # Format scores
                rmse, recall = analyse(data, s, 1, j)
                temp2 = pd.DataFrame(rmse)
                temp3 = temp2.transpose()
                temp3.columns = columns
                results_rmse = results_rmse.append(temp3)
                    
                temp2 = pd.DataFrame(recall)
                temp3 = temp2.transpose()
                temp3.columns = columns2
                results_recall = results_recall.append(temp3)
                
            results_rmse.index = index
            results_recall.index = index
                
            temp = pd.DataFrame(0, index=['Average'], columns=columns)
            for n in results_rmse.columns:
                temp.loc['Average', n] = results_rmse[n].mean()
            results_rmse = pd.concat([results_rmse, temp])
            
            temp = pd.DataFrame(0, index=['Average'], columns=columns2)
            for n in results_recall.columns:
                temp.loc['Average', n] = results_recall[n].mean()
            results_recall = pd.concat([results_recall, temp])
            
            # Save stores to csv file
            results_rmse.to_csv(str(i) + str(j) + 'rmse_over_mean.csv')
            results_recall.to_csv(str(i) + str(j) + 'recall_over_mean.csv')

In [ ]:
def analyse_pca(cbs_data, seed, over, j):
    # This function is very similar to the regular analyse function. Except it only covers KNN and RF while
    # including PCA
    
    random.seed(seed)
    
    x_columns = ['NL_ACHT', 'WE_ACHT', 'NW_ACHT', 'EENP', 'MPZK', 'EENOUD', 'TWEEOUD', 'HH_GR', 'STED', 'MAN',
              'L014', 'L1425', 'L2544', 'L4564', 'H045', 'H4564', 'H6574', 'H7584', 'H8594', 'H9504', 'H0514']

    y_column = ['inef_bin']

    X = cbs_data[x_columns]
    # X = pd.get_dummies(X)
    y = cbs_data[y_column]
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    # Reduce size of efficient class through undersampling
    if (over == 0) and (j != 0.5):
        X_train = pd.DataFrame(X_train)
        X_train.index = y_train.index
        y_train_inef = y_train[y_train['inef_bin'] == 1]
        y_train_ef = y_train[y_train['inef_bin'] == 0]
        X_train_inef = X_train.loc[y_train_inef.index, :]

        #Undersampling
        from random import sample 

        # Prints list of random items of given length 
        list1 = set(y_train_ef.index)

        samp = sample(list1, len(y_train_inef))

        y_train_ef = y_train.loc[samp, :]
        X_train_ef = X_train.loc[samp, :]
        
        y_train = pd.concat([y_train_ef, y_train_inef])
        X_train = pd.concat([X_train_ef, X_train_inef])
    
    # Increase size of inefficient class through oversampling
    if (over == 1) and (j != 0.5):
        # transform the dataset
        X_train, y_train = SMOTE().fit_sample(X_train, y_train)
        
    pca = decomposition.PCA(n_components=7)
    pca.fit(X_train)
    X_train = pca.transform(X_train)
    X_test = pca.transform(X_test)
    
    # KNN
    neigh = [1, 2, 3, 4, 5, 6, 7]
    random_grid = {'n_neighbors': neigh}

    knn_random = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = random_grid, cv = 5, verbose=0, n_jobs = -1)

    knn_random.fit(X_train, y_train)

    knn_predictions = knn_random.predict(X_test)
    
    # SVM
    C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    gammas = [0.001, 0.01, 0.1, 1, 10, 100]
    
    # Create the random grid
    random_grid = {'C': C,'gamma': gammas}

    SVM_random = GridSearchCV(estimator = SVC(), param_grid = random_grid, cv = 5, verbose=0, n_jobs = -1)
    
    SVM_random.fit(X_train, y_train)
    svm_predictions = SVM_random.predict(X_test)
    
    # RF
    rf = RandomForestClassifier(n_estimators = 100)
    rf.fit(X_train, y_train)
    rf_predictions = rf.predict(X_test)
    
    # NN
    mlp_gs = MLPClassifier(max_iter=100)
    
    import itertools
    x = [0, 5, 10, 20]
    sizes = [p for p in itertools.product(x, repeat=4)]
    
    for i in range(len(sizes)):
        p = list(sizes[i])
        while 0 in p:
            p.remove(0)
            sizes[i] = p
    
    parameter_space = {
        'hidden_layer_sizes': sizes,
        'activation': ['relu'],
        'solver': ['lbfgs'],
        'alpha': [1e-5],
        'learning_rate': ['constant'],
    }
    
    clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
    clf.fit(X, y)
    
    nn_predictions = clf.predict(X_test)
    
    # Create accuracy scores for algorithms above
    trues = list(y_test['inef_bin'])

    errors = [abs(a_i - b_i) for a_i, b_i in zip(knn_predictions, trues)]

    rmse = [math.sqrt(x**2) for x in errors]

    knn_rmse = 1 - sum(rmse)/len(rmse)

    errors = [abs(a_i - b_i) for a_i, b_i in zip(rf_predictions, trues)]

    rmse = [math.sqrt(x**2) for x in errors]

    rf_rmse = 1 - sum(rmse)/len(rmse)

    errors = [abs(a_i - b_i) for a_i, b_i in zip(svm_predictions, trues)]

    rmse = [math.sqrt(x**2) for x in errors]

    svm_rmse = 1 - sum(rmse)/len(rmse)

    errors = [abs(a_i - b_i) for a_i, b_i in zip(nn_predictions, trues)]

    rmse = [math.sqrt(x**2) for x in errors]

    nn_rmse = 1 - sum(rmse)/len(rmse)
    
    # Create recall scores for all 4 algorithms
    from sklearn.metrics import recall_score
    knn_rec = recall_score(trues, knn_predictions)
    rf_rec = recall_score(trues, rf_predictions)
    svm_rec = recall_score(trues, svm_predictions)
    nn_rec = recall_score(trues, nn_predictions)
    
    # Create accuracy scores if only predicting 0 or 1
    preds = [1] * len(y_test)
    trues = y_test[y_column]
    trues = list(trues.iloc[:, 0])
    
    cols = ['trues', 'preds']
    errors = pd.DataFrame(columns=cols)
    errors['trues'] = trues
    errors['preds'] = preds
    
    errors['errors'] = errors['trues'] - errors['preds']

    errors['errors'] = errors['errors'].apply(lambda x: math.sqrt(x**2))
    
    true_rmse = 1 - sum(errors['errors'])/len(errors)
    false_rmse = 1 - true_rmse
    
    return [round(knn_rmse, 2), round(rf_rmse, 2), round(svm_rmse, 2), round(nn_rmse, 2), round(false_rmse, 2), round(true_rmse, 2)], [round(knn_rec, 2), round(rf_rec, 2), round(svm_rec, 2), round(nn_rec, 2)]

In [ ]:
# Identical to previous section, except incorporating PCA
for i in cut:
    if i == 0.5:
        continue
    for j in rik:
            print(i, j)
            
            print('Normaal')
            
            columns = ['KNN (PCA)', 'RF (PCA)', '0', '1']
            columns2 = ['KNN (PCA)', 'RF (PCA)']
            
            results_rmse = pd.DataFrame(columns=columns)
            results_recall = pd.DataFrame(columns=columns2)
            
            for s in seeds:
                data = create_combi(rep_data.copy(), cbs_data.copy(), i, j)

                rmse, recall = analyse_pca(data, s, 10, j)
                temp2 = pd.DataFrame(rmse)
                temp3 = temp2.transpose()
                temp3.columns = columns
                results_rmse = results_rmse.append(temp3)
                    
                temp2 = pd.DataFrame(recall)
                temp3 = temp2.transpose()
                temp3.columns = columns2
                results_recall = results_recall.append(temp3)
            
            results_rmse.index = index
            results_recall.index = index
            
            temp = pd.DataFrame(0, index=['Average'], columns=columns)
            for n in results_rmse.columns:
                temp.loc['Average', n] = round(results_rmse[n].mean(), 2)
            results_rmse = pd.concat([results_rmse, temp])
            
            temp = pd.DataFrame(0, index=['Average'], columns=columns2)
            for n in results_recall.columns:
                temp.loc['Average', n] = round(results_recall[n].mean(), 2)
            results_recall = pd.concat([results_recall, temp])
            
            temp = pd.read_csv(str(i) + str(j) + 'rmse.csv')
            temp.index = results_rmse.index
            results_rmse['KNN'] = temp['KNN']
            results_rmse['RF'] = temp['RF']
            
            temp = pd.read_csv(str(i) + str(j) + 'recall.csv')
            temp.index = results_recall.index
            results_recall['KNN'] = temp['KNN']
            results_recall['RF'] = temp['RF']
            
            cols = ['KNN (PCA)', 'KNN', 'RF (PCA)', 'RF', '0', '1']
            results_rmse = results_rmse[cols]
            cols = ['KNN (PCA)', 'KNN', 'RF (PCA)', 'RF']
            results_recall = results_recall[cols]
            
            results_rmse.to_csv(str(i) + str(j) + 'rmse_pca.csv')
            results_recall.to_csv(str(i) + str(j) + 'recall_pca.csv')
            
            results_rmse = pd.DataFrame(columns=columns)
            results_recall = pd.DataFrame(columns=columns2)
            
            print('Under')
            for s in seeds:
        
                data = create_combi(rep_data.copy(), cbs_data.copy(), i, j)

                rmse, recall = analyse_pca(data, s, 0, j)
                temp2 = pd.DataFrame(rmse)
                temp3 = temp2.transpose()
                temp3.columns = columns
                results_rmse = results_rmse.append(temp3)
                    
                temp2 = pd.DataFrame(recall)
                temp3 = temp2.transpose()
                temp3.columns = columns2
                results_recall = results_recall.append(temp3)

            results_rmse.index = index
            results_recall.index = index                
                
            temp = pd.DataFrame(0, index=['Average'], columns=columns)
            for n in results_rmse.columns:
                temp.loc['Average', n] = round(results_rmse[n].mean(), 2)
            results_rmse = pd.concat([results_rmse, temp])
            
            temp = pd.DataFrame(0, index=['Average'], columns=columns2)
            for n in results_recall.columns:
                temp.loc['Average', n] = round(results_recall[n].mean(), 2)
            results_recall = pd.concat([results_recall, temp])
            
            temp = pd.read_csv(str(i) + str(j) + 'rmse_under.csv')
            temp.index = results_rmse.index
            results_rmse['KNN'] = temp['KNN']
            results_rmse['RF'] = temp['RF']
            
            temp = pd.read_csv(str(i) + str(j) + 'recall_under.csv')
            temp.index = results_recall.index
            results_recall['KNN'] = temp['KNN']
            results_recall['RF'] = temp['RF']
            
            cols = ['KNN (PCA)', 'KNN', 'RF (PCA)', 'RF', '0', '1']
            results_rmse = results_rmse[cols]
            cols = ['KNN (PCA)', 'KNN', 'RF (PCA)', 'RF']
            results_recall = results_recall[cols]
                
            results_rmse.to_csv(str(i) + str(j) + 'rmse_under_pca.csv')
            results_recall.to_csv(str(i) + str(j) + 'recall_under_pca.csv')
            
            results_rmse = pd.DataFrame(columns=columns)
            results_recall = pd.DataFrame(columns=columns2)
            
            print('Over')
            for s in seeds:
                data = create_combi(rep_data.copy(), cbs_data.copy(), i, j)

                rmse, recall = analyse_pca(data, s, 1, j)
                temp2 = pd.DataFrame(rmse)
                temp3 = temp2.transpose()
                temp3.columns = columns
                results_rmse = results_rmse.append(temp3)
                    
                temp2 = pd.DataFrame(recall)
                temp3 = temp2.transpose()
                temp3.columns = columns2
                results_recall = results_recall.append(temp3)
                
                
            results_rmse.index = index
            results_recall.index = index
                
            temp = pd.DataFrame(0, index=['Average'], columns=columns)
            for n in results_rmse.columns:
                temp.loc['Average', n] = round(results_rmse[n].mean(), 2)
            results_rmse = pd.concat([results_rmse, temp])
            
            temp = pd.DataFrame(0, index=['Average'], columns=columns2)
            for n in results_recall.columns:
                temp.loc['Average', n] = round(results_recall[n].mean(), 2)
            results_recall = pd.concat([results_recall, temp])
            
            temp = pd.read_csv(str(i) + str(j) + 'rmse_over.csv')
            temp.index = results_rmse.index
            results_rmse['KNN'] = temp['KNN']
            results_rmse['RF'] = temp['RF']
            
            temp = pd.read_csv(str(i) + str(j) + 'recall_over.csv')
            temp.index = results_recall.index
            results_recall['KNN'] = temp['KNN']
            results_recall['RF'] = temp['RF']
            
            cols = ['KNN (PCA)', 'KNN', 'RF (PCA)', 'RF', '0', '1']
            results_rmse = results_rmse[cols]
            cols = ['KNN (PCA)', 'KNN', 'RF (PCA)', 'RF']
            results_recall = results_recall[cols]

            results_rmse.to_csv(str(i) + str(j) + 'rmse_over_pca.csv')
            results_recall.to_csv(str(i) + str(j) + 'recall_over_pca.csv')